<a href="https://colab.research.google.com/github/elidonda2-web/Alisia-7B-it-/blob/main/Entra%C3%AEnement_Alisia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Avant de commencer exécute ce code:

In [ ]:
import time
time.sleep(60)


# Entraînement des modèles Alisia avec format chat_template

Ce notebook est utilisé pour entraîner nos futurs modèles Alisia. Comme discuté dans le groupe, nous allons utiliser le format **chat_template** pour nous permettre de bien évoluer et de rendre le modèle multimodal.

## 📝 Format chat_template

Le format chat_template standardise la structure des conversations. Voici un exemple :

**Format :**
```python
{
    "messages": [
        {"role": "system", "content": "Tu es Alisia, un assistant utile."},
        {"role": "user", "content": "Bonjour, comment ça va ?"},
        {"role": "assistant", "content": "Salut ! Je vais bien, merci. Et toi ?"},
        {"role": "user", "content": "Très bien aussi !"}
    ]
}
```

🌟 Multimodalité

La multimodalité signifie que le modèle peut comprendre et générer différents types de contenu : texte, images, audio. Cela permet des interactions plus riches comme décrire une image ou générer du contenu visuel à partir d'un texte.

🚀 Accélération de l'entraînement

Nous utilisons Unsloth pour rendre l'entraînement 2x plus rapide avec moins de mémoire utilisée.

📚 Bibliothèques à importer

Les bibliothèques principales à importer sont :

· unsloth pour l'accélération
· transformers pour les modèles et tokenizers
· datasets pour la gestion des données
· accelerate pour l'entraînement distribué

# voici le code:

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

## 🎯 Notre stratégie pour créer Alisia

### 🏗️ Une approche intelligente et efficace

Plutôt que de tout construire depuis zéro (ce qui prendrait des mois et coûterait très cher), nous utilisons une méthode éprouvée :

**Qwen2-7B est notre fondation de départ**
- C'est comme si on reprenait un cerveau déjà éduqué
- Il connaît déjà le français, l'anglais, la logique, le raisonnement
- On évite de réinventer la roue

### 🎨 Notre vraie valeur : personnaliser Alisia

**Ce qui va rendre Alisia unique, c'est NOTRE travail :**
- ✅ On va lui donner sa personnalité propre
- ✅ On va lui apprendre notre façon de communiquer
- ✅ On va l'entraîner sur nos sujets préférés
- ✅ On va modeler ses réponses selon notre style

### 🚀 Résultat final
Au bout du processus, Qwen2-7B ne sera plus reconnaissable. Vous parlerez avec **Alisia** - une intelligence unique qui portera notre empreinte.

**L'avantage** : On obtient rapidement un assistant performant, tout en gardant 100% de notre identité.

C'est la méthode utilisée par la majorité des projets d'IA aujourd'hui : partir d'une base solide pour construire quelque chose d'unique plus rapidement !

# voici le code:

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
# Try more models at https://huggingface.co/unsloth!

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Unsloth/Qwen2-7B", # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


# 🔧 Explication du code pour débutants

## 🎯 Ce que fait cette partie :

Cette fonction prépare notre modèle pour l'entraînement en utilisant une technique spéciale qui est **économe et efficace**.

---

## 📖 Explication ligne par ligne :

### **La base :**
```python
model = FastLanguageModel.get_peft_model(model, ...)
```

→ "Prends notre modèle Qwen2-7B et prépare-le pour un entraînement optimisé"

Les paramètres importants :

r = 16

· C'est le "niveau de personnalisation"
· Comme si on disait : "On va modifier 16 aspects du modèle pour lui donner la personnalité d'Alisia"
· Plus ce nombre est grand, plus la personnalisation est forte

target_modules = ["q_proj", "k_proj", ...]

· Ce sont les "parties du cerveau" qu'on va adapter
· Chaque module est comme une zone spécialisée :
  · q_proj, k_proj, v_proj = la compréhension du langage
  · gate_proj, up_proj, down_proj = la génération des réponses
· On cible les zones les plus importantes pour la conversation

lora_alpha = 16

· C'est "l'intensité" des modifications
· Comme le volume de nos ajustements

lora_dropout = 0

· Désactive une fonction optionnelle pour plus de stabilité

bias = "none"

· On garde les réglages originaux du modèle (plus simple)

use_gradient_checkpointing = "unsloth"

· 🚀 La magie d'Unsloth !
· Réduit la mémoire utilisée de 30%
· Permet de traiter 2x plus de données à la fois
· = Entraînement plus rapide et moins cher

---

💡 En résumé :

Cette technique (appelée LoRA) nous permet de :

· ✅ Personnaliser le modèle pour créer Alisia
· ✅ Économiser énormément de mémoire et de temps
· ✅ Garder toutes les connaissances de base de Qwen2-7B
· ✅ Ajouter seulement notre touche personnelle

C'est comme si on ajoutait une "couche de personnalité" par-dessus l'intelligence existante, sans tout réécrire !

Résultat : On obtient Alisia plus rapidement, avec moins de ressources ! 🎉

# voici le code:

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # ✅ Augmenté à 32 pour plus de personnalisation mais on peut toujours revenir à 16. Je sais pourquoi.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32, # ✅ Ajusté en conséquence
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.10.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.



# 🔧 Explication du code : Préparation des données

## 🎯 Ce que fait cette partie :

Cette section prépare nos conversations pour que le modèle puisse les comprendre et apprendre d'elles.

---

## 📖 Explication ligne par ligne :

### **Chargement des données**
```python
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
```

→ "Télécharge 100,000 conversations d'entraînement depuis internet"

Standardisation (Nettoyage et unification)

```python
dataset = standardize_sharegpt(dataset)
```

→ Pourquoi on fait ça ?
Ce dataset particulier (FineTome-100k) contient des conversations qui viennent de différentes sources et qui peuvent avoir des formats variés.
La standardisation les transforme toutes dans le même format standard, comme si on prenait des livres écrits avec différentes polices et marges, et qu'on les réimprimait tous de la même façon.

Sans standardisation : ❌

· Certaines conversations seraient illisibles pour le modèle
· Le modèle ne saurait pas où trouver les questions et réponses
· L'entraînement serait désordonné

#Avec standardisation : ✅

· Toutes les conversations ont la même structure
· Le modèle comprend facilement qui parle et quand
· L'apprentissage est plus efficace

Le format de conversation (Chat Template)

```python
tokenizer.chat_template = """..."""
```

→ "On définit comment Alisia doit se présenter et comment structurer les conversations"

Exemple de ce que ça produit :

```
<|im_start|>system
You are Alisia, a helpful assistant...<|im_end|>
<|im_start|>user
Bonjour !<|im_end|>
<|im_start|>assistant
Salut ! Comment puis-je t'aider ?<|im_end|>
```

La fonction de formatage

```python
def formatting_prompts_func(examples):
```

→ "Prend chaque conversation et la transforme dans le format qu'Alisia comprend"

Application du formatage

```python
dataset = dataset.map(formatting_prompts_func, batched=True)
```

→ "On applique la transformation à TOUTES les 100,000 conversations, par lots pour aller plus vite"

---

#Important à comprendre :

La standardisation peut varier selon le dataset :

#Pourquoi ça change :

· Chaque dataset a ses particularités
· Certains datasets sont déjà bien organisés, d'autres non
· On adapte le nettoyage selon les besoins

#Ce qui reste constant :

· Le besoin d'avoir des données bien organisées
· L'importance d'un format cohérent que nous utiliserons
· La préparation pour un bon apprentissage

---

#En résumé :

On standardise parce que ce dataset en a besoin pour être utilisable. C'est comme ranger et nettoyer des ingrédients avant de cuisiner : essentiel pour un bon résultat ! 🍳

#voici le code:

In [4]:
# --- 📘 Préparation du Dataset et du Chat Template (avec message système) ---

from datasets import load_dataset
from unsloth import standardize_sharegpt

# 1️⃣ Charger le dataset
print("🔹 Chargement du dataset...")
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

# 2️⃣ Standardiser le format pour obtenir 'conversations'
print("🔹 Standardisation du dataset...")
dataset = standardize_sharegpt(dataset)

# 3️⃣ Définir ton propre Chat Template (ChatML + message système par défaut)
tokenizer.chat_template = """<|im_start|>system
You are Alisia, a helpful, precise, and knowledgeable assistant created by the Alisia Research Team.<|im_end|>
{% for message in messages %}
<|im_start|>{{ message['role'] }}
{{ message['content'] }}<|im_end|>
{% endfor %}
{% if add_generation_prompt %}<|im_start|>assistant
{% endif %}"""


# 4️⃣ Fonction pour formater les conversations
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,  # Ne pas ajouter <|im_start|>assistant à la fin
        )
        for convo in convos
    ]
    return {"text": texts}

# 5️⃣ Appliquer le formatage sur tout le dataset
print("🔹 Application du formatage des conversations...")
dataset = dataset.map(formatting_prompts_func, batched=True)

# ✅ Vérification rapide d’un exemple
print("✅ Exemple de texte formaté :\n")
print(dataset[5]["text"])

🔹 Chargement du dataset...


README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

🔹 Standardisation du dataset...


Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

🔹 Application du formatage des conversations...


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

✅ Exemple de texte formaté :

<|im_start|>system
You are Alisia, a helpful, precise, and knowledgeable assistant created by the Alisia Research Team.<|im_end|>
<|im_start|>user
How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|im_end|>
<|im_start|>assistant
Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|im_end|>




# 🏋️ Explication du code : Configuration de l'entraînement

## 🎯 Ce que fait cette partie :

Cette section configure et lance l'entraînement d'Alisia avec nos données préparées.

---

## 📖 Explication simple :

```python
trainer = SFTTrainer(...)
```

→ "On crée un coach intelligent qui va entraîner Alisia"

Les paramètres principaux :

· model = model → Le modèle Qwen2-7B qu'on va personnaliser
· tokenizer = tokenizer → Le traducteur qui comprend nos conversations
· train_dataset = dataset → Nos 100,000 conversations préparées
· max_steps = 60 → On fait 60 cycles d'entraînement (pas trop pour commencer)
· learning_rate = 2e-4 → La vitesse d'apprentissage (ni trop lent, ni trop rapide)
· per_device_train_batch_size = 2 → 2 conversations traitées en même temps

---

#En résumé :

Ce code lance l'entraînement où Alisia apprend à partir de nos conversations. Le coach ajuste progressivement sa personnalité pour qu'elle réponde comme on le souhaite !

Prochaine étape : Une fois cet entraînement terminé, Alisia aura sa propre personnalité ! 🚀

##voici le code:

In [5]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

##on peut lancer l'entraînement et sa peut prendre des minutes selon:

**la connexion:** vérifier sans arrêt votre connexion mobile.


**la taille de datase:** plus le dataset est élevé, plus vous devrez patienter.

##Voici le code de lancement:

In [6]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 80,740,352 of 7,696,356,864 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.974100
2,0.866800
3,1.162600
4,0.878400
5,0.933200
6,0.825700
7,0.623200
8,0.895500
9,0.728000
10,0.617700


##♡ Mieux vaut prévenir que guérir on doit envoyer le fichier dans drive tout d'abord on va enregistrer le fichier qui contient le nouveau poids de modèle.

In [7]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

##Ensuite:

In [ ]:
# assure toi que ton drive qu'on tient >5G de stockage

# 🗻 Sauvegarde vers Google Drive

from google.colab import drive
import shutil
import os

# Monter Google Drive
print("📁 Montage de Google Drive...")
drive.mount('/content/drive')

# Chemin du modèle LoRA et destination
lora_model_path = "/content/lora_model"  # Chemin actuel
drive_backup_path = "/content/drive/MyDrive/lora_model"  # Chemin dans Drive

# Vérifier si le modèle LoRA existe
if os.path.exists(lora_model_path):
    print(f"✅ Modèle LoRA trouvé dans : {lora_model_path}")

    # Créer le dossier de sauvegarde dans Drive
    os.makedirs(drive_backup_path, exist_ok=True)

    # Copier tout le dossier vers Drive
    print("📤 Copie du modèle vers Google Drive...")
    shutil.copytree(lora_model_path, drive_backup_path, dirs_exist_ok=True)

    print("🎉 **SAUVEGARDE RÉUSSIE !**")
    print(f"📂 Modèle sauvegardé dans : {drive_backup_path}")
    print(f"📊 Taille du dossier : {sum(os.path.getsize(os.path.join(dirpath, filename)) for dirpath, dirnames, filenames in os.walk(lora_model_path) for filename in filenames) / (1024*1024):.2f} MB")

else:
    print("❌ ERREUR : Dossier lora_model non trouvé !")
    print("💡 Vérifie que l'entraînement s'est bien terminé")

# Vérification finale
if os.path.exists(drive_backup_path):
    print("\n🔍 **VÉRIFICATION :**")
    files = os.listdir(drive_backup_path)
    print(f"📄 Fichiers sauvegardés : {len(files)}")
    for file in files:
        print(f"   - {file}")
    print("\n✅ **Modèle Alisia en sécurité dans Google Drive !**")

📁 Montage de Google Drive...
Mounted at /content/drive
✅ Modèle LoRA trouvé dans : /content/lora_model
📤 Copie du modèle vers Google Drive...
🎉 **SAUVEGARDE RÉUSSIE !**
📂 Modèle sauvegardé dans : /content/drive/MyDrive/alisia_lora_model
📊 Taille du dossier : 323.19 MB

🔍 **VÉRIFICATION :**
📄 Fichiers sauvegardés : 10
   - tokenizer.json
   - tokenizer_config.json
   - adapter_model.safetensors
   - chat_template.jinja
   - special_tokens_map.json
   - merges.txt
   - README.md
   - adapter_config.json
   - added_tokens.json
   - vocab.json

✅ **Modèle Alisia en sécurité dans Google Drive !**


#Si vous avez encore de la mémoire vous pouvez tester si sa fonctionne:

In [ ]:

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Quel est la capitale de la France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1208,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

##NB: sa permet juste d'avoir les adapters LoRA pas le modèle complète faite de recherche sur internet et pour comprendre c'est quoi LoRA.